# Integrated model for routing pollsters and vehicles

<div class="alert alert-block alert-info">
This notebook contains the code to build and solve the Integrated Vehicles and Polsters Routing Problem (IVPRP). 
</div>

In [1]:
from gurobipy import *

In [2]:
# Packages
import numpy  as np
import pandas as pd
import time
from collections import deque

In [3]:
# Aliases
append, arange, around, asarray, loadtxt, zeros = np.append, np.arange, np.around, np.asarray, np.loadtxt, np.zeros
DataFrame, concat = pd.DataFrame, pd.concat

In [4]:
# General functions for reading and writing
def Reader(nom):    return loadtxt('Instances/'+ nom + '.txt' )
def P_Reader(n):    return  Reader('Pollster_' + str(n))
def V_Reader(n):    return  Reader( 'Vehicle_' + str(n))
def S_Reader(n):    return  Reader( 'Service_' + str(n))
def T_Reader(n):    return  Reader(    'Time_' + str(n))

def Write(df,nom):  return DataFrame(df).to_csv('Instances/'+ nom +'.txt', header=None, index=None, sep=' ', mode='a')

## Basics

<div class="alert alert-block alert-warning">
First, basic data is registered:
    
* $n$ is the number of stores to be visited,
* $E$ is the set of available pollsters,
* $K$ is the set of available vehicles,
* $Q$ is the vehicle's capacity,
* $S$ is the number of days.
</div>

In [5]:
n = 20
E = arange(5);    K = arange(3);    Q = min(1,E.size);    S = arange(15)

<div class="alert alert-block alert-success">
Now we read the data associated with the instance.
</div>

In [6]:
Service  = S_Reader(n)
Poll     = P_Reader(n)
Vehicles = V_Reader(n)
Time     = T_Reader(n)

<div class="alert alert-block alert-warning">
The read matrices are processed:
    
* $d$ captures pollstering times,
* $t$ is the time that a pollster takes to walk among pairs of stores,
* $\tau$ is the time that vehicles take to move between pairs of stores,

* $[\rho_0,\rho_1]$ is the time window for breaks,
* $P$ is the length of the pause,
* $\beta$ is the time horizon limit.
</div>

In [7]:
d = append(around(Service, 2), 0.0)
t = around(Poll, 2)
τ = around(Vehicles, 2)
ρ_0, ρ_1, P, β = Time.ravel()

<div class="alert alert-block alert-warning">
At last costs are fixed:
    
* $\kappa_0$ is the daily cost of operations,
* $\kappa_1$ is the cost of hiring a vehicle,
* $\kappa_2$ is the cost of hiring a pollster.
</div>

In [8]:
κ_0 = 200.0
κ_1 = 100.0
κ_2 = 40.0

Finding an upper bound:

\begin{align}
    &\min_{(e\in E, k\in K, s\in S)} \kappa_0 (s+1) + \kappa_1 (k+1)(s+1) + \kappa_{2} (e+1)(s+1)
    \\
    \text{subject to} \qquad &
    \\
    & \dfrac{\left( \sum_{i\in C_-} t_{i,i+n} + (e+1) \times (s+1) \times P \right)}{(e+1)\times (s+1)} + \dfrac{ \left( \sum \max_{i} \tau_{} + \max_{j} \tau \right) }{ (k+1) \times (s+1) } \leq B_{\max}
    \\
    & (e+1) \leq Q (k+1)
\end{align}

In [9]:
Feas = {(e+1,k+1,s+1): (d.sum() + (e+1)*(s+1)*P)/((e+1)*(s+1)) 
            + (τ.max(axis=0) + τ.max(axis=1)).sum()/((k+1)*(s+1)*(1)) for e in E for k in K for s in S}
#{(e+1,k+1,s+1): (d.sum() + (e+1)*(s+1)*P)/((e+1)*(s+1)) + τ.sum()/((k+1)*(s+1)*(e+2)) for e in E for k in K for s in S}
#print({aa: (around(bb,2), bb < β) for aa,bb in Feas.items()})
print('')
Feas = {aa: κ_2 * aa[0] * aa[2] + κ_1 * aa[1] * aa[2] + κ_0 * aa[2] for aa,bb in Feas.items() if (bb < β) and (aa[1] <= aa[0] <= Q * (aa[1]))}
mE, mK, mS = min(Feas, key=Feas.get)
#print(Feas)
print('\nAt most there should be',mE,'pollsters,',mK,'vehicles, and',mS,'days.')



At most there should be 2 pollsters, 2 vehicles, and 2 days.


In [10]:
E = arange(mE);    K = arange(mK);    S = arange(mS)

## Read partition

<div class="alert alert-block alert-warning">
We read each partition and reduce the graph size.
</div>

In [11]:
ϵ = 2
Partition = pd.read_csv('Instances/'+ 'Partition_'+ str(n) + '_' + str(ϵ) + '.txt', 
                delimiter = ' ', header = None, skip_blank_lines=True, 
                skiprows  = [aa for aa in range(14) if aa not in [4,7,10,13]], 
                   engine = 'python',  index_col=False)

part_0 = Partition.iloc[0,:].dropna().values.astype(int)
part_1 = Partition.iloc[1,:].dropna().values.astype(int)
if ϵ > 2:
    part_2 = Partition.iloc[2,:].dropna().values.astype(int)
    part_3 = Partition.iloc[3,:].dropna().values.astype(int)
display(DataFrame(part_0.reshape(1,-1)))
display(DataFrame(part_1.reshape(1,-1)))
if ϵ > 2:
    display(DataFrame(part_2.reshape(1,-1)))
    display(DataFrame(part_3.reshape(1,-1)))

,0,1,2,3,4,5,6,7,8,9,10
0,1,2,3,6,7,8,9,12,14,17,19


,0,1,2,3,4,5,6,7,8
0,4,5,10,11,13,15,16,18,20


Take one part:

In [12]:
part = part_0.copy();    N = part.size

In [13]:
d_p = d[[0]+list(part)+[-1]]
t_p = t[part-1][:,part-1]
τ_p = τ[[0]+list(part)][:,[0]+list(part)]
α   = 0.5*(β - P)

Find new upper bounds (these might fail, yet as the bounds are general, it is still possible to find a feasible solution):

In [14]:
Feas = {(e+1,k+1): d_p.sum()/(e+1) + (τ_p.max(axis=0) + τ_p.max(axis=1)).sum()/(k+1) for e in E for k in K}
Feas = {aa: κ_2 * aa[0] + κ_1 * aa[1] for aa,bb in Feas.items() if (bb < α) and (aa[1] <= aa[0] <= Q * (aa[1]))}
if len(Feas) > 0:
    mE, mK = min(Feas, key=Feas.get)
    print('Tighter bounds found.')
    print('At most there should be',mE,'pollsters and',mK,'vehicles.')
else:
    print('No tighter bounds found, the instance might be infeasible.')

No tighter bounds found, the instance might be infeasible.


In [15]:
E = arange(mE);    K = arange(mK)

## Subgraph of $N$ points from actual graph

<div class="alert alert-block alert-warning">
The multigraph is built.
</div>

In [16]:
m = 2*n+1

# Indices
C_minus, C_plus, C_0, C = part.copy(), part + n, append(0,part), append(part,part + n);    C_m = append(C,m)

# Domain for vehicles.                         Graph size:    (2*N) * (N + 1) + (2*N-1) * N = 4*(N**2) + N
dom_v  = [ (0,j)   for j in C ]
dom_v += [ (j,m)   for j in C ]
dom_v += [ (i,j)   for i in C for j in C if j not in [i,i-n]]

# Domain for pollsters.                        Graph size:    (N-1)**2 + (N-1) + N = N**2
dom_e  = [ (i,i+n) for i in C_minus ]
dom_e += [ (i,j)   for i in C_plus for j in C_minus if j!=i-n ]

## Integer Programming Model

<div class="alert alert-block alert-warning">
Now we are ready to build the IP model for IVPRP.
</div>

In [17]:
mo = Model()
x, y, z, b, f, w, B, u = {}, {}, {}, {}, {}, {}, {}, {}

Using license file /Users/andy/gurobi.lic
Academic license - for non-commercial use only


### Variables and Objective

In [18]:
# Pollster variables
## Nodes
b = mo.addVars( C_minus, E, vtype = 'B', name ='b')             # **begining**    N * |E|
f = mo.addVars( C_plus,  E, vtype = 'B', name ='f')             # **ending**      N * |E|
## Arcs
x = mo.addVars( dom_e, E, vtype = 'B', name = 'x')              # **Walking paths**       N^2 * |E|

# Vehicle variables
y = mo.addVars( dom_v, K, vtype = 'B', name = 'y')               # **Vehicle-paths**     (4*N^2+N)*|K|
z = mo.addVars( dom_v, E, vtype = 'B', name = 'z')               # **t-paths**           (4*N^2+N)*|E|

# Time and days variables
B = mo.addVars( C, vtype = 'C', name = 'B', ub  = α )               # **In-Out timing**     2*N

mo.update()

Additional terms in objective

In [19]:
deque( (v.setAttr('obj', κ_1) for v in y.select(0,'*')), 0)
deque( (v.setAttr('obj', κ_2) for v in z.select(0,'*')), 0)
mo.setAttr('ModelSense', GRB.MINIMIZE)

In [20]:
mo.update()

### Constraints

<div class="alert alert-block alert-warning">
We begin with constraints for pollster routing, these are constraints $(1a)$ to $(1h)$.
</div>

In [21]:
# x & z vars interaction
start = time.time()
# 1a - Exclusive attention:      sum x[i,i+n] = 1
mo.addConstrs( (x.sum(i,i+n,'*') == 1 for i in C_minus), name='R-1a');

# 1b,c - Flow conservation:          sum_j x[j,i] - x[i,j] = b[i] - f[i]
mo.addConstrs( (x.sum('*',i,e) == x[i,i+n,e] - b[i,e] for i in C_minus for e in E), name='R-1b')
mo.addConstrs( (x.sum(i,'*',e) == x[i-n,i,e] - f[i,e] for i in C_plus  for e in E), name='R-1c')

# 1d,e,f,g — Terminal pick-up and delivery:  sum_j z[i,j] <= 1
mo.addConstrs( (z.sum('*',i,e) <= 1.0 - x[i,i+n,e] + b[i,e] for i in C_minus for e in E), name='R-1d')
mo.addConstrs( (z.sum(i,'*',e) <= 1.0 - x[i-n,i,e] + f[i,e] for i in C_plus  for e in E), name='R-1e')

mo.addConstrs( (z.sum('*',i,e) - z.sum(i,'*',e) ==  b[i,e] for i in C_minus for e in E), name='R-1f')
mo.addConstrs( (z.sum('*',i,e) - z.sum(i,'*',e) == -f[i,e] for i in C_plus  for e in E), name='R-1g')

# Additional valid bounds
mo.addConstrs( (z.sum('*',i,e) <= 1.0 for i in C for e in E), name='R-1d-add')
mo.addConstrs( (z.sum(i,'*',e) <= 1.0 for i in C for e in E), name='R-1d-add')

# 1h – One trip per day for each pollster
mo.addConstrs( (z.sum(0,'*',e) <= 1 for e in E), name='R-1h')


end = time.time()
print('First block took {} seconds to compute.'.format(end-start))

mo.update()

First block took 0.12082219123840332 seconds to compute.


<div class="alert alert-block alert-warning">
Then we turn to vehicle routing, these are constraints $(2a)$ to $(2f)$.
</div>

In [22]:
# y & z vars interaction
start = time.time()
# 2a,b - Terminal arrivals:       sum_(j,k) y[i,j] = sum_e b[i] + f[i]
mo.addConstrs( (y.sum(i,'*','*') == b.sum(i,'*') for i in C_minus), name='R-2a')
mo.addConstrs( (y.sum(i,'*','*') == f.sum(i,'*') for i in C_plus ), name='R-2b')
# 2c,d – Flow conservation:       sum_j y[i,j] - sum_j y[j,i] = 0
mo.addConstrs( (y.sum('*',i,k)     == y.sum(i,'*',k) for i in C for k in K ), name='R-2c')
mo.addConstrs( (y.sum('*',2*n+1,k) == y.sum(0,'*',k) for k in K ), name='R-2d')
# 2e – One trip per day for each vehicle:     sum_i y[0,i] <= 1
mo.addConstrs( (y.sum(0,'*',k) <= 1.0 for k in K ), name='R-2e')
# 2f – Capacity load:             sum_e z[i,j] <= Q sum_k y[i,j]
mo.addConstrs( (z.sum(i,j,'*') <= Q*y.sum(i,j,'*') for (i,j) in dom_v ), name='R-2f')

end = time.time()
print('Took {} seconds.'.format(end-start))

mo.update()

Took 0.21680998802185059 seconds.


<div class="alert alert-block alert-warning">
Next are time management and shift lengths $(3a)$ to $(3e)$.
</div>

In [23]:
M = α + max(τ_p.max(), t.max()) + 10.#1e+4

In [24]:
# B vars enforce connected paths
start = time.time()
# 3a,b – Arriving marker:  B[j] >= B[i] + t[i,j] + sum_s w[i] P - M(1 - sum_s x[i,j])
mo.addConstrs( (B[i+n] - B[i] - d[i] >= 0.0 for i in C_minus ), name='R-3a')
mo.addConstrs( (B[j] - B[i+n] - t[i-1,j-1] >= -M*(1.0 - x.sum(i+n,j,'*')) 
               for i in C_minus for j in C_minus if j!=i ), name='R-3b')
## Trivial
mo.addConstrs( (B[i+n] - B[i] >= 0.0 for i in C_minus ), name='R-3-o')
# 3c — Arrival after transport: B[j] >= B[i] + τ_{i,j} - M(1 - sum_{s,k} y[i,j] )
mo.addConstrs( 
    (B[j] - B[i] + M*(1.0 - y.sum(i,j,'*')) >= τ[i % (n+1) + (1 if i > n else 0), j % (n+1) + (1 if j > n else 0)]
     for (i,j) in dom_v if i!=0 and j!=2*n+1 ), name='R-3c')
# 3d — First transportation: B[i] >= tau_{0,i} - M(1 - sum_{s,k} y[0,i]^{k,s} )
mo.addConstrs( (B[i] >= τ[0, i % (n+1) + (1 if i > n else 0)] - α*(1.0 - y.sum(0,i,'*','*')) 
               for i in C ), name='R-3d')

# 3e — Arrival marks: B[2n+1]^{s} >= B[i] + tau_{i,2n+1} - M(1-y[i,2n+1])
mo.addConstrs( ( α - B[i] >= τ[i % (n+1) + (1 if i > n else 0),0] - M*(1.0 - y.sum(i,2*n+1,'*'))
               for i in C), name='R-3e')
# 3f – There's no TW information available.
end = time.time()
print('Took {} seconds.'.format(end-start))

mo.update()

Took 0.2230072021484375 seconds.


### Symmetry breaking inequalities

<div class="alert alert-block alert-warning">
Shorter versions of constraints $(5a)$ to $(5l)$ are added here.
</div>

In [25]:
# More
start = time.time()

mo.addConstr( z.sum(0,'*',0) == 1 , name='R-5a')
mo.addConstr( y.sum(0,'*',0) == 1 , name='R-5d')

if E.size > 1:
    mo.addConstrs( (z.sum(0,'*',e) <= z.sum(0,'*',e-1) for e in E if e > 0 ), name='R-5b')
    mo.addConstrs( (b[i,e]   <= 1.0 - b[i,e-1] for i in C_minus for e in E if e > 0 ), name='R-5k' )
    mo.addConstrs( (f[i,e]   <= 1.0 - f[i,e-1] for i in C_plus  for e in E if e > 0 ),  name='R-5l' )
    mo.addConstrs( (x[i,j,e] <= 1.0 - x[i,j,e-1] for (i,j) in dom_e for e in E if e > 0 ), name='R-5m');
    
if K.size > 1:
    mo.addConstrs( (y.sum(0,'*',k) <= y.sum(0,'*',k-1) for k in K if k > 0 ), name='R-5e')
    mo.addConstrs( (y[i,j,k] <= 1.0 - y[i,j,k-1] for (i,j) in dom_v for k in K if k > 0 ), name='R-5o');
    
end = time.time()
print('SI: Took {} seconds.'.format(end-start))

SI: Took 0.10819292068481445 seconds.


In [26]:
# Valid inequalities
mo.addConstr( quicksum( B[i+n] - B[i] for i in C_minus ) >= d_p.sum(), "VB-b")
mo.addConstr( quicksum( B[i+n] - B[i] for i in C_minus ) <= E.size*α, "VB-6a");

---

In [27]:
Λ   = [ np.append(a, arange(a.size + b + 1, (b+1)*a.size + 1) ) for b,a in enumerate([(E+1)]) ]
Λ_β = {λ: d_p.sum()/λ for λ in np.unique(np.concatenate(Λ)) if d_p.sum()/λ <= α}
print(Λ_β)

{2: 44.0}


In [28]:
{λ: d_p.sum()/λ for λ in np.unique(np.concatenate(Λ)) if (d_p.sum())/λ <= α - (τ_p[0,:][1:].min() + τ_p[:,0][1:].min())}

{2: 44.0}

In [29]:
F_2, T_1 = max(Λ_β.items(), key=lambda x: x[1]) 
F_2, T_1

(2, 44.0)

In [30]:
F_3 = [k+1 for k in K if F_2 <= Q * (k+1)].pop(0)
F_3

2

In [31]:
# Additional bound over the number of required pollsters
mo.addConstr(z.sum(0,'*','*','*') >= F_2);
# Additional bound over the number of required vehicles
mo.addConstr(y.sum(0,'*','*','*') >= F_3);

In [32]:
deque( (v.setAttr('BranchPriority', 10) for v in y.values()), 0);
#deque( (v.setAttr('BranchPriority', 10) for v in z.values()), 0)

## Optimization

Pick the best solution (if any) from below.

In [33]:
mo.reset()
mo.resetParams()
mo.Params.VarBranch = 1
mo.Params.BranchDir = 0
mo.Params.TimeLimit = 360
#mo.Params.Cuts = 0
mo.optimize()

Discarded solution information
Reset all parameters
Changed value of parameter VarBranch to 1
   Prev: -1  Min: -1  Max: 3  Default: -1
Parameter BranchDir unchanged
   Value: 0  Min: -1  Max: 1  Default: 0
Changed value of parameter TimeLimit to 360.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.0.2 build v9.0.2rc0 (mac64)
Optimize a model with 2071 rows, 2288 columns and 14388 nonzeros
Model fingerprint: 0x11d762bb
Variable types: 22 continuous, 2266 integer (2266 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+02]
  Objective range  [4e+01, 1e+02]
  Bounds range     [1e+00, 7e+01]
  RHS range        [1e+00, 2e+02]
Presolve removed 67 rows and 11 columns
Presolve time: 0.11s
Presolved: 2004 rows, 2277 columns, 13944 nonzeros
Variable types: 22 continuous, 2255 integer (2255 binary)

Root relaxation: objective 2.800000e+02, 1194 iterations, 0.31 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  

In [34]:
Γ = {λ: d_p.sum()/λ for λ in np.unique(np.concatenate(Λ)) 
                 if (d_p.sum())/λ <= α - (τ_p[0,:][1:].min() + τ_p[:,0][1:].min())}
if Γ != Λ_β:
    F_2, T_1 = max(Γ.items(), key=lambda x: x[1]) 
    print('Bound can be tightened with',F_2,'pollsters!')
    mo.addConstr(z.sum(0,'*','*','*') >= F_2);

In [ ]:
mo.Params.TimeLimit = 360
mo.optimize()

In [ ]:
# Parametrised
mo.reset()
mo.resetParams()
#mo.setParam('Symmetry', 2) #mo.setParam('BranchDir', 1); mo.setParam('Heuristics',1)

mo.Params.MIPFocus = 1;    
mo.Params.Heuristics = 0.33
mo.Params.Cuts = 3;      #<- w/o finds some feasible solutions 
mo.Params.Method = 2

mo.Params.SimplexPricing = 3
mo.Params.CutAggPasses = 12;    mo.Params.CutPasses = 12;   mo.Params.PrePasses = 8

mo.Params.ImproveStartTime = 100


mo.setParam('Presolve', 2)
mo.Params.GURO_PAR_PREPROBE = 3
mo.Params.PreSparsify = 1
mo.Params.PrePasses = 500 # best solution so far w/ 500

mo.Params.TimeLimit = 10*60
mo.optimize()

## Results

<div class="alert alert-block alert-warning">
We identify variables that have assigned a non-zero value.
</div>

In [34]:
X = tupledict({nn: v for nn, v in x.items() if v.x > 0.0})
Y = tupledict({nn: v for nn, v in y.items() if v.x > 0.0})
Z = tupledict({nn: v for nn, v in z.items() if v.x > 0.0})

A = tupledict({nn: v for nn, v in b.items() if v.x > 0.0})
F = tupledict({nn: v for nn, v in f.items() if v.x > 0.0})

Active pollsters and vehicles are first identified. This simplifies iterations.

In [35]:
Active_Pollsters = {v[1] for v in A.keys()}
Active_Vehicles  = {v[2] for v in Y.keys()}

In [36]:
print('A solution was found with',len(Active_Pollsters),'pollsters and',len(Active_Vehicles),'vehicles.')

A solution was found with 2 pollsters and 2 vehicles.


In [37]:
Out_File = 'Instances/Results_Partition-'+ str(n) +'-2-a.xlsx'

In [38]:
with pd.ExcelWriter(Out_File) as writer:
    # General results
    Hoja = DataFrame({'0':['Active pollsters','Active vehicles', 'Objective','GAP','Time'], 
                      '1':[len(Active_Pollsters), len(Active_Vehicles), 
                               mo.ObjVal, str(around(mo.MIPGap * 100,2)) + ' %', around(mo.Runtime,2)]})
    Hoja.to_excel(writer, 'Summary', header=False, index= False)
    writer.sheets['Summary'].set_column('A:A', 15)
    
    # Pollster routing
    X_visits = zeros([n+1,n+1], dtype=int)
    for pairs in X.keys():
        coords = tuple([vv if vv <= n else vv%n if vv<2*n else n for vv in pairs[:-1] ])
        X_visits[coords] = pairs[-1] + 1
            
    Hoja = DataFrame(X_visits).replace({0:''})
    Hoja.columns = [str(v) for v in range(0,n+1) ]
    Hoja.to_excel(writer, 'Pollster routing')
    
    # Vehicle routing
    Y_visits = zeros([n+1,n+1], dtype=int)
    for pairs in Y.keys():
        coords = tuple([vv if vv <= n else vv%n if vv<2*n else n if vv == 2*n else 0 for vv in pairs[:-1] ])
        Y_visits[coords] = pairs[-1] + 1
    
    Hoja = DataFrame(Y_visits).replace({0:''})
    Hoja.columns = [str(v) for v in range(0,n+1) ]
    Hoja.to_excel(writer, 'Vehicle routing')
    
    # Shared routing
    Z_visits = zeros([n+1,n+1], dtype='<U'+str(2*n))
    for pairs in Z.keys():
        coords = tuple([vv if vv <= n else vv%n if vv<2*n else n if vv == 2*n else 0 for vv in pairs[:-1] ])
        if Z_visits[coords] == '':
            Z_visits[coords]  = str(pairs[-1] + 1)
        else:
            Z_visits[coords] += ', ' + str(pairs[-1] + 1)
    
    Hoja = DataFrame(Z_visits).replace({0:''})
    Hoja.columns = [str(v) for v in range(0,n+1) ]
    Hoja.to_excel(writer, 'Shared routing')
    
    # Times    
    Hoja = DataFrame({'Time i':[ B[i].x for i in C_minus ], 'Time i+n':[ B[i].x for i in C_plus ]})
    Hoja.index = C_minus
    Hoja.to_excel(writer, 'Times')

In [39]:
β, ρ_0, ρ_1, P, α

(100.0, 40.0, 60.0, 10.0, 45.0)

$a$

In [39]:
'''Actual scale'''
if mo.status != 3 and mo.objVal < 1e+4:
    for v in mo.getVars():
        if v.x > 1e-5:
            print('%s,%s.lb = %g' % (v.varName[:-1], '0]', v.x))

b[4,1,0].lb = 1
b[5,0,0].lb = 1
b[9,0,0].lb = 1
f[15,0,0].lb = 1
f[16,1,0].lb = 1
f[19,0,0].lb = 1
x[4,14,1,0].lb = 1
x[5,15,0,0].lb = 1
x[6,16,1,0].lb = 1
x[8,18,1,0].lb = 1
x[9,19,0,0].lb = 1
x[14,8,1,0].lb = 1
x[18,6,1,0].lb = 1
y[0,4,1,0].lb = 1
y[0,5,0,0].lb = 1
y[16,21,1,0].lb = 1
y[19,21,0,0].lb = 1
y[4,16,1,0].lb = 1
y[5,15,0,0].lb = 1
y[9,19,0,0].lb = 1
y[15,9,0,0].lb = 1
z[0,4,1,0].lb = 1
z[0,5,0,0].lb = 1
z[16,21,1,0].lb = 1
z[19,21,0,0].lb = 1
z[15,9,0,0].lb = 1
B[4,0].lb = 4.23
B[5,0].lb = 2.55
B[6,0].lb = 34.61
B[8,0].lb = 18.64
B[9,0].lb = 21.2
B[14,0].lb = 5.23
B[15,0].lb = 18.55
B[16,0].lb = 43.61
B[18,0].lb = 28.64
B[19,0].lb = 22.2


In [41]:
'''Actual scale'''
if mo.status != 3 and mo.objVal < 1e+4:
    for v in mo.getVars():
        if v.x > 1e-5:
            print('%s,%s.lb = %g' % (v.varName[:-1], '1]', v.x))

b[4,0,1].lb = 1
b[6,0,1].lb = 1
b[9,1,1].lb = 1
b[11,0,1].lb = 1
b[12,1,1].lb = 1
f[16,0,1].lb = 1
f[18,0,1].lb = 1
f[21,1,1].lb = 1
f[23,0,1].lb = 1
f[24,1,1].lb = 1
x[4,16,0,1].lb = 1
x[6,18,0,1].lb = 1
x[9,21,1,1].lb = 1
x[11,23,0,1].lb = 1
x[12,24,1,1].lb = 1
y[0,4,1,1].lb = 1
y[0,9,0,1].lb = 1
y[18,25,1,1].lb = 1
y[24,25,0,1].lb = 1
y[4,21,1,1].lb = 1
y[6,24,0,1].lb = 1
y[9,16,0,1].lb = 1
y[11,23,0,1].lb = 1
y[12,18,1,1].lb = 1
y[16,11,0,1].lb = 1
y[21,12,1,1].lb = 1
y[23,6,0,1].lb = 1
z[0,4,0,1].lb = 1
z[0,9,1,1].lb = 1
z[18,25,0,1].lb = 1
z[24,25,1,1].lb = 1
z[16,11,0,1].lb = 1
z[21,12,1,1].lb = 1
z[23,6,0,1].lb = 1
B[4,1].lb = 4.23
B[6,1].lb = 34.76
B[9,1].lb = 4.32
B[11,1].lb = 16.47
B[12,1].lb = 22.74
B[16,1].lb = 14.23
B[18,1].lb = 44
B[21,1].lb = 20.89
B[23,1].lb = 31.47
B[24,1].lb = 39.74


$b$

In [41]:
'''Actual scale'''
if mo.status != 3 and mo.objVal < 1e+4:
    for v in mo.getVars():
        if v.x > 1e-5:
            print('%s,%s.lb = %g' % (v.varName[:-1], '0]', v.x))

b[4,1,0].lb = 1
b[5,0,0].lb = 1
b[9,0,0].lb = 1
f[15,0,0].lb = 1
f[16,1,0].lb = 1
f[19,0,0].lb = 1
x[4,14,1,0].lb = 1
x[5,15,0,0].lb = 1
x[6,16,1,0].lb = 1
x[8,18,1,0].lb = 1
x[9,19,0,0].lb = 1
x[14,8,1,0].lb = 1
x[18,6,1,0].lb = 1
y[0,4,1,0].lb = 1
y[0,5,0,0].lb = 1
y[16,21,1,0].lb = 1
y[19,21,0,0].lb = 1
y[4,16,1,0].lb = 1
y[5,15,0,0].lb = 1
y[9,19,0,0].lb = 1
y[15,9,0,0].lb = 1
z[0,4,1,0].lb = 1
z[0,5,0,0].lb = 1
z[16,21,1,0].lb = 1
z[19,21,0,0].lb = 1
z[15,9,0,0].lb = 1
B[4,0].lb = 4.23
B[5,0].lb = 2.55
B[6,0].lb = 34.61
B[8,0].lb = 18.64
B[9,0].lb = 21.2
B[14,0].lb = 5.23
B[15,0].lb = 18.55
B[16,0].lb = 43.61
B[18,0].lb = 28.64
B[19,0].lb = 22.2


In [42]:
'''Actual scale'''
if mo.status != 3 and mo.objVal < 1e+4:
    for v in mo.getVars():
        if v.x > 1e-5:
            print('%s,%s.lb = %g' % (v.varName[:-1], '1]', v.x))

b[4,1,1].lb = 1
b[5,0,1].lb = 1
b[9,0,1].lb = 1
f[15,0,1].lb = 1
f[16,1,1].lb = 1
f[19,0,1].lb = 1
x[4,14,1,1].lb = 1
x[5,15,0,1].lb = 1
x[6,16,1,1].lb = 1
x[8,18,1,1].lb = 1
x[9,19,0,1].lb = 1
x[14,8,1,1].lb = 1
x[18,6,1,1].lb = 1
y[0,4,1,1].lb = 1
y[0,5,0,1].lb = 1
y[16,21,1,1].lb = 1
y[19,21,0,1].lb = 1
y[4,16,1,1].lb = 1
y[5,15,0,1].lb = 1
y[9,19,0,1].lb = 1
y[15,9,0,1].lb = 1
z[0,4,1,1].lb = 1
z[0,5,0,1].lb = 1
z[16,21,1,1].lb = 1
z[19,21,0,1].lb = 1
z[15,9,0,1].lb = 1
B[4,1].lb = 4.23
B[5,1].lb = 2.55
B[6,1].lb = 34.61
B[8,1].lb = 18.64
B[9,1].lb = 21.2
B[14,1].lb = 5.23
B[15,1].lb = 18.55
B[16,1].lb = 43.61
B[18,1].lb = 28.64
B[19,1].lb = 22.2


$c$

In [39]:
'''Actual scale'''
if mo.status != 3 and mo.objVal < 1e+4:
    for v in mo.getVars():
        if v.x > 1e-5:
            print('%s,%s.lb = %g' % (v.varName[:-1], '0]', v.x))

b[11,1,0].lb = 1
b[14,0,0].lb = 1
f[27,1,0].lb = 1
f[28,0,0].lb = 1
x[9,23,1,0].lb = 1
x[11,25,1,0].lb = 1
x[13,27,1,0].lb = 1
x[14,28,0,0].lb = 1
x[23,13,1,0].lb = 1
x[25,9,1,0].lb = 1
y[0,11,0,0].lb = 1
y[0,14,1,0].lb = 1
y[27,29,0,0].lb = 1
y[28,29,1,0].lb = 1
y[11,27,0,0].lb = 1
y[14,28,1,0].lb = 1
z[0,11,1,0].lb = 1
z[0,14,0,0].lb = 1
z[27,29,1,0].lb = 1
z[28,29,0,0].lb = 1
B[9,0].lb = 16.18
B[11,0].lb = 4.51
B[13,0].lb = 23.81
B[14,0].lb = 4.51
B[23,0].lb = 18.18
B[25,0].lb = 13.51
B[27,0].lb = 36.81
B[28,0].lb = 23.51


In [40]:
'''Actual scale'''
if mo.status != 3 and mo.objVal < 1e+4:
    for v in mo.getVars():
        if v.x > 1e-5:
            print('%s,%s.lb = %g' % (v.varName[:-1], '1]', v.x))

b[11,1,1].lb = 1
b[14,0,1].lb = 1
f[27,1,1].lb = 1
f[28,0,1].lb = 1
x[9,23,1,1].lb = 1
x[11,25,1,1].lb = 1
x[13,27,1,1].lb = 1
x[14,28,0,1].lb = 1
x[23,13,1,1].lb = 1
x[25,9,1,1].lb = 1
y[0,11,0,1].lb = 1
y[0,14,1,1].lb = 1
y[27,29,0,1].lb = 1
y[28,29,1,1].lb = 1
y[11,27,0,1].lb = 1
y[14,28,1,1].lb = 1
z[0,11,1,1].lb = 1
z[0,14,0,1].lb = 1
z[27,29,1,1].lb = 1
z[28,29,0,1].lb = 1
B[9,1].lb = 16.18
B[11,1].lb = 4.51
B[13,1].lb = 23.81
B[14,1].lb = 4.51
B[23,1].lb = 18.18
B[25,1].lb = 13.51
B[27,1].lb = 36.81
B[28,1].lb = 23.51


$d$

In [39]:
'''Actual scale'''
if mo.status != 3 and mo.objVal < 1e+4:
    for v in mo.getVars():
        if v.x > 1e-5:
            print('%s,%s.lb = %g' % (v.varName[:-1], '0]', v.x))

b[1,0,0].lb = 1
b[3,1,0].lb = 1
f[15,0,0].lb = 1
f[16,1,0].lb = 1
x[1,15,0,0].lb = 1
x[2,16,1,0].lb = 1
x[3,17,1,0].lb = 1
x[17,2,1,0].lb = 1
y[0,1,0,0].lb = 1
y[0,3,1,0].lb = 1
y[15,29,0,0].lb = 1
y[16,29,1,0].lb = 1
y[1,15,0,0].lb = 1
y[3,16,1,0].lb = 1
z[0,1,0,0].lb = 1
z[0,3,1,0].lb = 1
z[15,29,0,0].lb = 1
z[16,29,1,0].lb = 1
B[1,0].lb = 3.69
B[2,0].lb = 19.79
B[3,0].lb = 3.42
B[15,0].lb = 19.69
B[16,0].lb = 28.79
B[17,0].lb = 18.42


In [40]:
'''Actual scale'''
if mo.status != 3 and mo.objVal < 1e+4:
    for v in mo.getVars():
        if v.x > 1e-5:
            print('%s,%s.lb = %g' % (v.varName[:-1], '1]', v.x))

b[1,0,1].lb = 1
b[3,1,1].lb = 1
f[15,0,1].lb = 1
f[16,1,1].lb = 1
x[1,15,0,1].lb = 1
x[2,16,1,1].lb = 1
x[3,17,1,1].lb = 1
x[17,2,1,1].lb = 1
y[0,1,0,1].lb = 1
y[0,3,1,1].lb = 1
y[15,29,0,1].lb = 1
y[16,29,1,1].lb = 1
y[1,15,0,1].lb = 1
y[3,16,1,1].lb = 1
z[0,1,0,1].lb = 1
z[0,3,1,1].lb = 1
z[15,29,0,1].lb = 1
z[16,29,1,1].lb = 1
B[1,1].lb = 3.69
B[2,1].lb = 19.79
B[3,1].lb = 3.42
B[15,1].lb = 19.69
B[16,1].lb = 28.79
B[17,1].lb = 18.42


In [80]:
'''Actual scale'''
if mo.status != 3 and mo.objVal < 1e+4:
    for v in mo.getVars():
        if v.x > 1e-5:
            print('%s,%s.lb = %g' % (v.varName[:-1], '0]', v.x))

b[1,1,0].lb = 1
b[2,0,0].lb = 1
b[3,2,0].lb = 1
b[6,0,0].lb = 1
b[7,1,0].lb = 1
b[8,0,0].lb = 1
b[9,2,0].lb = 1
b[12,1,0].lb = 1
b[14,2,0].lb = 1
b[17,2,0].lb = 1
b[19,0,0].lb = 1
f[21,1,0].lb = 1
f[22,0,0].lb = 1
f[23,2,0].lb = 1
f[26,0,0].lb = 1
f[27,1,0].lb = 1
f[28,0,0].lb = 1
f[29,2,0].lb = 1
f[32,1,0].lb = 1
f[34,2,0].lb = 1
f[37,2,0].lb = 1
f[39,0,0].lb = 1
x[1,21,1,0].lb = 1
x[2,22,0,0].lb = 1
x[3,23,2,0].lb = 1
x[6,26,0,0].lb = 1
x[7,27,1,0].lb = 1
x[8,28,0,0].lb = 1
x[9,29,2,0].lb = 1
x[12,32,1,0].lb = 1
x[14,34,2,0].lb = 1
x[17,37,2,0].lb = 1
x[19,39,0,0].lb = 1
y[0,6,0,0].lb = 1
y[0,39,1,0].lb = 1
y[28,41,0,0].lb = 1
y[37,41,1,0].lb = 1
y[1,37,1,0].lb = 1
y[2,27,1,0].lb = 1
y[3,23,0,0].lb = 1
y[6,7,0,0].lb = 1
y[7,26,0,0].lb = 1
y[8,12,0,0].lb = 1
y[9,2,1,0].lb = 1
y[12,34,0,0].lb = 1
y[14,21,0,0].lb = 1
y[17,32,0,0].lb = 1
y[19,29,0,0].lb = 1
y[21,22,0,0].lb = 1
y[22,8,0,0].lb = 1
y[23,17,0,0].lb = 1
y[26,19,0,0].lb = 1
y[27,1,1,0].lb = 1
y[29,14,0,0].lb = 1
y[32,28,0,0].l

In [108]:
'''Actual scale'''
if mo.status != 3 and mo.objVal < 1e+4:
    for v in mo.getVars():
        if v.x > 1e-5:
            print('%s.start = %g' % (v.varName, v.x))

b[1,0].start = 1
b[2,0].start = 1
b[3,0].start = 1
b[4,0].start = 1
b[5,0].start = 1
b[6,0].start = 1
b[8,0].start = 1
b[10,0].start = 1
b[11,0].start = 1
f[12,0].start = 1
f[13,0].start = 1
f[14,0].start = 1
f[15,0].start = 1
f[16,0].start = 1
f[17,0].start = 1
f[19,0].start = 1
f[20,0].start = 1
f[22,0].start = 1
x[1,12,0].start = 1
x[2,13,0].start = 1
x[3,14,0].start = 1
x[4,15,0].start = 1
x[5,16,0].start = 1
x[6,17,0].start = 1
x[7,18,0].start = 1
x[8,19,0].start = 1
x[9,20,0].start = 1
x[10,21,0].start = 1
x[11,22,0].start = 1
x[18,9,0].start = 1
x[21,7,0].start = 1
y[0,3,0].start = 1
y[16,23,0].start = 1
y[1,12,0].start = 1
y[2,13,0].start = 1
y[3,14,0].start = 1
y[4,15,0].start = 1
y[5,16,0].start = 1
y[6,17,0].start = 1
y[8,19,0].start = 1
y[10,20,0].start = 1
y[11,22,0].start = 1
y[12,10,0].start = 1
y[13,8,0].start = 1
y[14,2,0].start = 1
y[15,11,0].start = 1
y[17,1,0].start = 1
y[19,4,0].start = 1
y[20,5,0].start = 1
y[22,6,0].start = 1
z[0,3,0].start = 1
z[16,23,0].start =

---